In [2]:

# data processing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# natural language processing
import nltk
nltk.data.path.append('/usr/share/nltk_data/')
from nltk import pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import string

from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# machine learning imports
from funcsigs import signature
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, average_precision_score, precision_recall_curve, roc_curve

# matplotlib things
plt.style.use('seaborn-v0_8-poster')

In [3]:
# import data
df = pd.read_csv('./data/combined_sentiments.csv', header=0, sep=',', on_bad_lines='skip')

# lemmatise
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# check whether there is a digit or not
def check_digits(text):
     return any(i.isdigit() for i in text)

# tokenise
def clean_review(review):
	review = str(review)
	review = review.lower() # turn into lowercase
	review = [word.strip(string.punctuation) for word in review.split(' ')] # remove punctuation
	review = [word for word in review if not check_digits(word)] # remove digits

	# remove stop words
	stop = stopwords.words('english')
	review = [token for token in review if token not in stop]
	# remove empty tokens
	review = [token for token in review if len(token) > 0]
	
	# tag each token with its part of speech (pos)
	pos_tags = pos_tag(review)
	review = [WordNetLemmatizer().lemmatize(tag[0], get_wordnet_pos(tag[1])) for tag in pos_tags]

	# remove words with only one letter
	review = [token for token in review if len(token) > 1]
	review = ' '.join(review)
	return review

# generate a cleaned, tokenised and lemmatised version of the reviews
df['reviews.clean'] = df['reviews.text'].apply(lambda x: clean_review(x))

reviews = df["reviews.clean"].values.tolist()
# print(reviews[:6])

"""
* create a list with all the unique words in the whole corpus of reviews.
* construct a feature vector that contains the counts of how often each word occurs
	in the review.
"""
count_vectoriser = CountVectorizer()
wordbag = count_vectoriser.fit_transform(reviews)
print(count_vectoriser.vocabulary_)

{'experience': 4278, 'rancho': 9515, 'valencia': 12737, 'absolutely': 35, 'perfect': 8728, 'begin': 1126, 'end': 3970, 'felt': 4479, 'special': 11135, 'happy': 5393, 'stay': 11318, 'would': 13391, 'come': 2355, 'back': 889, 'heart': 5478, 'beat': 1077, 'amazing': 384, 'place': 8904, 'everyone': 4174, 'extremely': 4327, 'warm': 13044, 'welcoming': 13148, 'we': 13111, 've': 12781, 'top': 12133, 'notch': 8085, 'definitely': 3144, 'great': 5203, 'romantic': 10146, 'getaway': 5036, 'take': 11714, 'kid': 6502, 'along': 352, 'couple': 2748, 'stuffed': 11464, 'animal': 465, 'wait': 12987, 'girl': 5057, 'upon': 12657, 'arrival': 639, 'can': 1727, 'go': 5105, 'book': 1368, 'night': 7980, 'play': 8937, 'tennis': 11847, 'since': 10806, 'one': 8275, 'high': 5561, 'rat': 9536, 'resort': 9905, 'america': 405, 'really': 9576, 'luxury': 7085, 'standpoint': 11276, 'overall': 8415, 'villa': 12880, 'staff': 11240, 'attention': 763, 'detail': 3286, 'include': 5936, 'fresh': 4820, 'squeeze': 11219, 'orange'

In [4]:
"""
raw term frequency: frequency of each token
[term frequency-inverse document frequency]
* tf - idf(t, d) = tf(t, d) idf(t, d)
> tf(t, d) is the raw term frequency;
> idf(t, d) is the inverse document frequency: log (n_d / [1 + df(d, t)])
	* n - total number of documents
  * df(t, d) - number of documents where term t appears.
"""

from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True,
                         norm='l2',
                         smooth_idf=True)

np.set_printoptions(precision=2)

# feed the tf-idf transformer with our previously created bag
tfidf.fit_transform(wordbag).toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
# split into train, test and validation sets
X = df['reviews.clean']
y = df['sent.polarity']
X_t, X_test, y_t, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_t, y_t, test_size=0.25, random_state=1, stratify=y_t)

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

# create a parameter grid for the model to pick the best params
paramgrid = [{'vect__ngram_range': [(1, 1)],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

# train the model!
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)
leftright = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

gridsearch = GridSearchCV(leftright, paramgrid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)
gridsearch.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/home/jiyo/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/jiyo/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
 

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)]},
                         {'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)], 'vect__norm': [None],
                          'vect__use_idf': [False]}],
             scoring='accuracy', verbose=1)

In [13]:
print('best accuracy: %.3f' % gridsearch.best_score_)

clf = gridsearch.best_estimator_
print('accuracy in test: %.3f' % clf.score(X_test, y_test))

best accuracy: 0.768
accuracy in test: 0.781


In [28]:
# make some predictions
print(type(X_val))
print(X_val.shape)
print(X_val.head())
print(X_val.iloc[:,1])

IndexError: tuple index out of range